# Policy Impacts for Year 2100

This notebook calculates static budgetary impacts for all reform options using the 2100 dataset.
This provides a test of how reforms perform in the long-term projection.

**Dataset**: `hf://policyengine/test/2100.h5`  
**Year**: 2100  
**Scoring**: Static (no behavioral responses)

In [7]:
# Import necessary libraries
import sys
import os

# Determine repo root and add src to path
if os.path.basename(os.getcwd()) == 'analysis':
    repo_root = os.path.abspath('..')
    os.chdir(repo_root)
else:
    repo_root = os.getcwd()

# Add src directory to Python path
src_path = os.path.join(repo_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"Working directory: {os.getcwd()}")
print(f"Source path: {src_path}")

import pandas as pd
import numpy as np
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from reforms import REFORMS
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"✓ Libraries imported")
print(f"✓ Found {len(REFORMS)} reforms")

Working directory: c:\Users\dtsax\PolicyEngine\crfb-tob-impacts
Source path: c:\Users\dtsax\PolicyEngine\crfb-tob-impacts\src
✓ Libraries imported
✓ Found 8 reforms


## Load 2100 Dataset

Load the 2100 projection dataset to test long-term impacts of reforms.

In [8]:
# Load the 2100 dataset
print("Loading 2100 dataset...")
sim100 = Microsimulation(dataset="hf://policyengine/test/2100.h5")
print("✓ Dataset loaded successfully")

# Check dataset size
household_weight = sim100.calculate("household_weight", period=2100)
household_count = sim100.calculate("household_count", period=2100, map_to="household")

print(f"\nDataset statistics:")
print(f"  Number of households in sample: {len(household_weight):,}")
print(f"  Weighted household count: {household_count.sum():,.0f}")

Loading 2100 dataset...
✓ Dataset loaded successfully

Dataset statistics:
  Number of households in sample: 21,108
  Weighted household count: 198,418,860


## Compute Baseline

Calculate baseline income tax for year 2100 using the 2100 dataset.

In [9]:
print("Computing baseline for 2100...")
baseline_2100 = Microsimulation(dataset="hf://policyengine/test/2100.h5")
baseline_income_tax = baseline_2100.calculate("income_tax", map_to="household", period=2100)

print(f"✓ Baseline computed")
print(f"  Total baseline income tax: ${baseline_income_tax.sum() / 1e9:,.1f}B")

Computing baseline for 2100...
✓ Baseline computed
  Total baseline income tax: $1,999.0B


## Helper Function

Define function to calculate revenue impact for a given reform.

In [10]:
def calculate_revenue_impact_2100(reform):
    """
    Calculate revenue impact for a given reform in year 2100.
    
    Args:
        reform: Reform object
    
    Returns:
        Revenue impact in dollars (positive = revenue gain, negative = revenue loss)
    """
    # Create reformed simulation with 2100 dataset
    reform_sim = Microsimulation(dataset="hf://policyengine/test/2100.h5", reform=reform)
    
    # Calculate reformed income tax
    reform_income_tax = reform_sim.calculate("income_tax", map_to="household", period=2100)
    
    # JCT convention: reformed - baseline (positive = more revenue)
    revenue_impact = reform_income_tax.sum() - baseline_income_tax.sum()
    
    return revenue_impact

print("✓ Helper function defined")

✓ Helper function defined


## Calculate Reform Impacts for 2100

Test all 8 reform options with the 2100 dataset.

In [11]:
# Storage for results
results_2100 = []

print("\n" + "="*80)
print("CALCULATING REFORM IMPACTS FOR YEAR 2100")
print("="*80)
print(f"Testing {len(REFORMS)} reforms\n")

for reform_id, reform_config in tqdm(REFORMS.items(), desc="Processing reforms"):
    reform_name = reform_config['name']
    reform_func = reform_config['func']
    
    print(f"\nProcessing {reform_id}: {reform_name}")
    
    try:
        # Get the reform
        reform = reform_func()
        
        # Calculate impact
        print(f"  Calculating 2100 impact...", end=' ')
        impact = calculate_revenue_impact_2100(reform)
        
        results_2100.append({
            'reform_id': reform_id,
            'reform_name': reform_name,
            'year': 2100,
            'revenue_impact': impact,
            'revenue_impact_billions': impact / 1e9,
            'scoring_type': 'static',
            'dataset': '2100.h5'
        })
        
        print(f"${impact/1e9:,.1f}B")
        print(f"  ✓ Complete")
        
    except Exception as e:
        print(f"  ✗ ERROR: {type(e).__name__}: {e}")
        print(f"  Continuing with next reform...")
        import traceback
        traceback.print_exc()

print("\n" + "="*80)
print("CALCULATION COMPLETE")
print("="*80)


CALCULATING REFORM IMPACTS FOR YEAR 2100
Testing 8 reforms



Processing reforms:   0%|          | 0/8 [00:00<?, ?it/s]


Processing option1: Full Repeal of Social Security Benefits Taxation
  Calculating 2100 impact...   ✗ ERROR: MemoryError: Unable to allocate 209. KiB for an array with shape (53430,) and data type float32
  Continuing with next reform...


Traceback (most recent call last):
  File "C:\Users\dtsax\AppData\Local\Temp\ipykernel_16656\3534950836.py", line 21, in <module>
    impact = calculate_revenue_impact_2100(reform)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\AppData\Local\Temp\ipykernel_16656\2169566121.py", line 15, in calculate_revenue_impact_2100
    reform_income_tax = reform_sim.calculate("income_tax", map_to="household", period=2100)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\simulations\microsimulation.py", line 54, in calculate
    values = super().calculate(variable_name, period, map_to, decode_enums)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dtsax\PolicyEngine\policyengine-core\policyengine_core\simulations\simulation.py", line 485, in calculate
    result = self._calculate(variable_name, period)
             ^


Processing option2: Taxation of 85% of Social Security Benefits
  Calculating 2100 impact... 

Processing reforms:  12%|█▎        | 1/8 [02:53<20:14, 173.56s/it]


KeyboardInterrupt: 

## Summary of Results

In [ ]:
# Convert to DataFrame
results_df = pd.DataFrame(results_2100)

if len(results_df) > 0:
    print("\n2100 Reform Impacts (Billions):")
    print("="*80)
    
    for _, row in results_df.iterrows():
        print(f"{row['reform_id']:8s}: {row['reform_name']:55s} ${row['revenue_impact_billions']:>8,.1f}B")
    
    print("="*80)
    print(f"\nTotal reforms calculated: {len(results_df)}")
    
    # Display as table
    display(results_df[['reform_id', 'reform_name', 'revenue_impact_billions']].sort_values('revenue_impact_billions', ascending=False))
else:
    print("⚠ No results to display")

## Export Results to CSV

Save the 2100 impact estimates to a CSV file for further analysis.

In [ ]:
# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

if len(results_df) > 0:
    # Export full results
    output_file = 'data/policy_impacts_2100.csv'
    results_df.to_csv(output_file, index=False)
    print(f"✓ Exported results to: {output_file}")
    print(f"  Records: {len(results_df)}")
    print(f"  Columns: {', '.join(results_df.columns)}")
    
    # Also create a summary version
    summary_df = results_df[['reform_id', 'reform_name', 'revenue_impact_billions']].copy()
    summary_df = summary_df.sort_values('revenue_impact_billions', ascending=False)
    summary_file = 'data/policy_impacts_2100_summary.csv'
    summary_df.to_csv(summary_file, index=False)
    print(f"✓ Exported summary to: {summary_file}")
else:
    print("⚠ No results to export")

print("\n✓ Analysis complete!")